In [ ]:
import os
gpu_num = "" # Use "" to use the CPU
os.environ["CUDA_VISIBLE_DEVICES"] = f"{gpu_num}"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

resolution = [480,320] # increase for higher quality of renderings

# Allows to exit cell execution in Jupyter
class ExitCell(Exception):
    def _render_traceback_(self):
        pass

# Import Sionna
try:
    import sionna
except ImportError as e:
    # Install Sionna if package is not already installed
    import os
    os.system("pip install sionna")
    import sionna

# Configure the notebook to use only a single GPU and allocate only as much memory as needed
# For more details, see https://www.tensorflow.org/guide/gpu
import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        print(e)
# Avoid warnings from TensorFlow
tf.get_logger().setLevel('ERROR')

tf.random.set_seed(1) # Set global random seed for reproducibility

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np
import time

# Import Sionna RT components
from sionna.rt import load_scene, Transmitter, Receiver, PlanarArray, RadioMaterial, LambertianPattern
from sionna.constants import PI

# For link-level simulations
from sionna.channel import cir_to_ofdm_channel, subcarrier_frequencies, OFDMChannel, ApplyOFDMChannel, CIRDataset
from sionna.nr import PUSCHConfig, PUSCHTransmitter, PUSCHReceiver
from sionna.utils import compute_ber, ebnodb2no, PlotBER
from sionna.ofdm import KBestDetector, LinearDetector
from sionna.mimo import StreamManagement

In [ ]:
# Load integrated scene
scene = load_scene("../../scenes/fraunhofer_full_5_percent_aligned/fraunhofer_full_5_percent_aligned.xml")

In [ ]:
scene.get("elm__10").radio_material = "itu_concrete" # base, if it's not been mapped it is of this material 
scene.get("elm__11").radio_material = "itu_wood" # tree: options are also itu_very_dry_ground, itu_medium_dry_ground, itu_wet_ground
scene.get("elm__12").radio_material = "itu_concrete" # building
scene.get("elm__13").radio_material = "itu_glass" # window
scene.get("elm__14").radio_material = "itu_medium_dry_ground" # grass
scene.get("elm__15").radio_material = "itu_metal" # vehicles
scene.get("elm__16").radio_material = "itu_concrete" # stree

In [ ]:
scene.preview()